### GHZ SIZE 16 IN IBM (Falcon) AND QUANTINUUM

In [1]:
# Caller
from pathlib import Path
import sys

# Discover project root
proj = Path.cwd().resolve()
for _ in range(8):
    if (proj / "results" / "pairwise" / "summary").exists() and (proj / "notebooks").exists():
        break
    if proj.parent == proj:
        break
    proj = proj.parent

# Import helper
sys.path.insert(0, str(proj / "notebooks"))
from notebook_helper import load_family, tidy, show_table  # (sort_best_worst, add_rank_within_size available if needed)

# analysis parameters
FAMILY = "ghz"  # or: "qaoa","vqe_two_local","qft","grover","vbe_adder","randomcircuit"
BACKENDS = ("ibm_falcon","quantinuum")

loaded = load_family(FAMILY, BACKENDS)

for backend, df in loaded.items():
    tbl = tidy(df)
    show_table(tbl, caption=f"{FAMILY.upper()} – {backend} (raw tidy)")

    # To rank later, uncomment:
    # from notebook_helper import sort_best_worst, add_rank_within_size
    # PRIMARY_METRIC = "depth_delta"  # or "n_ops_total_gates_delta", "n_ops_total_delta", etc.
    # ranked = sort_best_worst(tbl, primary=PRIMARY_METRIC, per_size=True)
    # ranked = add_rank_within_size(ranked, by=PRIMARY_METRIC)
    # show_table(ranked, caption=f"{FAMILY.upper()} – {backend} (ranked by {PRIMARY_METRIC})")

,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,ibm_falcon,ghz,16,-,RB→RR,19,19,+0,35,34,-1,18,18,+0,19,19,+0,15,15,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1], [""barrier"", 1]]","[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1]]",16,16,2025-10-05T19:53:04
1,ibm_falcon,ghz,16,-,RR→RB,19,19,+0,35,34,-1,18,18,+0,19,19,+0,15,15,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1], [""barrier"", 1]]","[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1]]",16,16,2025-10-05T19:53:04
2,ibm_falcon,ghz,16,-,RB→CTM,19,19,+0,35,34,-1,18,18,+0,19,19,+0,15,15,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1], [""barrier"", 1]]","[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1]]",16,16,2025-10-05T19:53:04
3,ibm_falcon,ghz,16,-,CTM→RB,19,19,+0,35,34,-1,18,18,+0,19,19,+0,15,15,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1], [""barrier"", 1]]","[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1]]",16,16,2025-10-05T19:53:04
4,ibm_falcon,ghz,16,-,RR→CTM,19,19,+0,35,35,+0,18,18,+0,19,19,+0,15,15,+0,1,1,+0,1,1,+0,16,16,+0,0,0,+0,"[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1], [""barrier"", 1]]","[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1], [""barrier"", 1]]",16,16,2025-10-05T19:53:04
5,ibm_falcon,ghz,16,-,CTM→RR,19,19,+0,35,35,+0,18,18,+0,19,19,+0,15,15,+0,1,1,+0,1,1,+0,16,16,+0,0,0,+0,"[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1], [""barrier"", 1]]","[[""measure"", 16], [""cx"", 15], [""rz"", 2], [""sx"", 1], [""barrier"", 1]]",16,16,2025-10-05T19:53:04


,backend,family,size,variant,sequence,depth_before,depth_after,depth_delta,n_ops_total_before,n_ops_total_after,n_ops_total_delta,n_ops_total_gates_before,n_ops_total_gates_after,n_ops_total_gates_delta,n_ops1_before,n_ops1_after,n_ops1_delta,n_ops2_before,n_ops2_after,n_ops2_delta,other_before,other_after,other_delta,barrier_before,barrier_after,barrier_delta,measure_before,measure_after,measure_delta,reset_before,reset_after,reset_delta,top_ops_before,top_ops_after,n_qubits_before,n_qubits_after,timestamp
0,quantinuum,ghz,16,-,RB→RR,63,48,-15,124,93,-31,107,77,-30,108,78,-30,15,15,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rx"", 46], [""ry"", 31], [""measure"", 16], [""zzphase"", 15], [""rz"", 15], [""barrier"", 1]]","[[""ry"", 31], [""rx"", 31], [""measure"", 16], [""zzphase"", 15]]",16,16,2025-10-05T19:53:04
1,quantinuum,ghz,16,-,RR→RB,63,48,-15,124,108,-16,107,92,-15,108,93,-15,15,15,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rx"", 46], [""ry"", 31], [""measure"", 16], [""zzphase"", 15], [""rz"", 15], [""barrier"", 1]]","[[""ry"", 31], [""rx"", 31], [""measure"", 16], [""zzphase"", 15], [""rz"", 15]]",16,16,2025-10-05T19:53:04
2,quantinuum,ghz,16,-,RB→CTM,63,78,+15,124,123,-1,107,107,+0,108,108,+0,15,15,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rx"", 46], [""ry"", 31], [""measure"", 16], [""zzphase"", 15], [""rz"", 15], [""barrier"", 1]]","[[""rx"", 46], [""ry"", 31], [""measure"", 16], [""rz"", 15], [""zzphase"", 15]]",16,16,2025-10-05T19:53:04
3,quantinuum,ghz,16,-,CTM→RB,63,78,+15,124,123,-1,107,107,+0,108,108,+0,15,15,+0,1,0,-1,1,0,-1,16,16,+0,0,0,+0,"[[""rx"", 46], [""ry"", 31], [""measure"", 16], [""zzphase"", 15], [""rz"", 15], [""barrier"", 1]]","[[""rx"", 46], [""ry"", 31], [""measure"", 16], [""rz"", 15], [""zzphase"", 15]]",16,16,2025-10-05T19:53:04
4,quantinuum,ghz,16,-,RR→CTM,63,63,+0,124,109,-15,107,92,-15,108,93,-15,15,15,+0,1,1,+0,1,1,+0,16,16,+0,0,0,+0,"[[""rx"", 46], [""ry"", 31], [""measure"", 16], [""zzphase"", 15], [""rz"", 15], [""barrier"", 1]]","[[""ry"", 31], [""rx"", 31], [""measure"", 16], [""rz"", 15], [""zzphase"", 15], [""barrier"", 1]]",16,16,2025-10-05T19:53:04
5,quantinuum,ghz,16,-,CTM→RR,63,63,+0,124,109,-15,107,92,-15,108,93,-15,15,15,+0,1,1,+0,1,1,+0,16,16,+0,0,0,+0,"[[""rx"", 46], [""ry"", 31], [""measure"", 16], [""zzphase"", 15], [""rz"", 15], [""barrier"", 1]]","[[""ry"", 31], [""rx"", 31], [""measure"", 16], [""rz"", 15], [""zzphase"", 15], [""barrier"", 1]]",16,16,2025-10-05T19:53:04


### quick findings

- **IBM (falcon)**  
  - `RB` always removes a `barrier`  
  - No pass changes depth across the RB/RR/CTM pairs 
  - `n_ops_total` decreases by **1** when `RB` runs : this equals `barrier_delta = -1`. All other metrics (1q, 2q, measures, resets, n_qubits) remain unchanged

- **Quantinuum**  
  - `RB ↔ RR` reduces **depth by 15** in both directions.  
  - **Order matters.** `RB → RR` reduces `n_ops_total` by **31** while `RR → RB` reduces by **16**. Breakdown for `RB → RR`:
    - `n_ops_total_delta = -31`
    - `n_ops_1q_delta = -30` (major simplification of 1-qubit gates)
    - `barrier_delta = -1`
    - `measures` remain (16)
  - `RB ↔ CTM` generally **adds depth** (worse for depth) and only removes the `barrier` (no 1q/2q change)
  - `RR ↔ CTM` is depth-neutral but reduces `n_ops_total` by **15** (all coming from 1-qubit gates) and does **not** remove the barrier

### interpretation 
`barrier` is a **compiler directive** (counts as an operation in `n_ops_total`) but **not** a physical gate. Removing a barrier often **unlocks** more aggressive algebraic simplifications (cancellations/merges) in later passes. That explains the asymmetric effect when `RB` is first vs. later.

---

### next step
- run 3-pass permutations
- observe ionq (forte) gate set too